In [ ]:
%matplotlib inline
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import scipy
from scipy import stats
import pandas as pd
from collections import Counter

# problem 1 

In [ ]:
a=[10., 10.3, 2.1, 0., 0., 15.6, 22.3, 12.7]
data=stats.norm.rvs(loc = 12., scale = 0.4, size = 100000)
full = np.append(data, a)
fig, ax = plt.subplots(1, 1)
ax.hist(d,100, density=True)
plt.xlabel("temperature reading")
plt.ylabel("Probability in log")
plt.yscale('log')
#x = np.linspace(4.95,5.05,1000)
#ax.plot(x,stats.norm.pdf(x,loc = 5., scale = 0.01),linewidth = 8,alpha = 0.7)
plt.show()

A2:QUESTION : What data point are within 5σ of the background distribution? The reason I choose this question is because this is the real data we keep that doesn't contain any anomalous read.

A3:Calculate the data inside the 5σ

A4:Truth table:

In [ ]:
X = np.sort(full)
D = np.array([])
D_1 = np.array([])

def truthtable(s):
    sigma = stats.norm.sf(s)
    maxTemp = stats.norm.isf(sigma, loc=12, scale=0.4)
    minTemp = stats.norm.isf(1-sigma, loc=12, scale=0.4)
    D = X[np.logical_and(X > minTemp, X < maxTemp)]
    D_1 = X[np.logical_or(X < minTemp, X > maxTemp)]
    TP = sum((Counter(data) & Counter(D)).values())
    FP = sum((Counter(D) & Counter(a)).values())
    TN = sum((Counter(data) & Counter(D_1)).values())
    FN = sum((Counter(D_1) & Counter(a)).values())
    
    table = pd.DataFrame(data=[[TP,FP],[TN,FN]],
                     index=["Pos", "Neg"],
                     columns=["True", "False"])
    return table

In [ ]:
truthtable(5)

There are two mistakes(12.7 and 10.3) , if we included all data at greater than 5 sigma significance there would be more FP less TN less FN

B:Counstruct truth table for different threshold

In [ ]:
truthtable(3)


When we choose 3, TP decresed by one and TN increase by one, however we need to thrown out 264 values

In [ ]:
truthtable(7)

When we choose 7, TP increased by one and TN decreased by one(increase errors of commission and reduce the errors of omission)so we can have more real data.

C:contruct a true table for all thrown out 12.7

In [ ]:
sigma = stats.norm.isf(stats.norm.sf(12.7, scale=0.4, loc=12))
truthtable(sigma)
sigma


We need to thrown out 8019 of real data in order to remove all mistakes of commission and about 1.75 sigma

2.


"What recorded temperature value on the thermal device is five sigmas or more from the mean so that we can get rid of these points of data? In more detail, at which x value of temperature in the new distribution (with 'bad data') is the probability of getting the temperature larger than the probability of five sigma to the right of the mean or less than the probability of five sigma to the left of the mean in a normal distribution?" We do this by finding the integral under the distribution we analyze at x that yields the same or less probability than 5 sigma on a normal distribution. We could search for five sigma of our new dataset by searching all values larger or less than the mean +- stdev * 5 but this doesn't neccessarily give us the probability of five sigma in a normal distribution which is what we're looking for. This method only works for a normal distribution and for any other distribution it would not work. This is my question because I am essentially asking "Which data points have a probability larger or less than 5 sigma?" in order to determine which of the data points are "bad data" and thus worthy of removing. We are using 5 sigma as a base line because the probability of 5 sigma or higher is about 1 in 3.5 million which is reasonable cause to see the data point as an outlier. We also use this baseline because if we go any lower we may be recieving too many false positives and thus losing out on good data. There is discussion to be had whether its better to have more false positives but less true negative or vice versa but we decided it is better to have less false positives and more true negatives.

3.

In [ ]:
fiveSigma = 1 - stats.norm.cdf(5)
xTop = stats.norm.ppf(1 - fiveSigma, mu, std)
xBot = stats.norm.ppf(fiveSigma, mu, std)
print(xTop)
print(xBot)

for i in reversed(range(len(d))):
    if d[i] > xTop:
        d = np.delete(d, i)
    if d[i] < xBot:
        d = np.delete(d, i)
print(len(d))
plt.hist(d, 100, density=True)
plt.yscale('log')
plt.title('Distribution of cleaned data')
plt.xlabel('Temperature (k)')
plt.ylabel('Probability')
plt.show()
x = list(range(len(d)))
plt.title('Plot of cleaned data points with respect to their index values')
plt.xlabel('Index value')
plt.ylabel('Value')
plt.plot(x, d)


First we calculated the 5 sigma probability of a normal distribution and then we calculated the ppf, of a normal distribution with the adjusted mean of 12 and standard deviation of .4. 

B

The statistical threshold determines the number of sigmas away from the mean. This affects the number of omissions greatly and deciding this cutoff is a game of deciding what number of false positives vs true negatives you are comfortable with. For example, if you want to make sure you are keeping all the good dat' you might want to have a larger sigma cut off so you are including more values, thus making sure to keep the good values. However, this might come at the cost of creating more false positives because the sensitivity to deviation is weaker and so more bad data may be left in the distribution. On the other hand, if we want to make sure you are getting rid of all of the bad data you would decrease the sigma cutoff so that you can make sure more of your bad data on the edges of the distribution are caught and removed. However, this comes at the cost of removing more good data as the sensitivity to deviation is much higher and some good data will be mistaken for bad data. Thus a general rule is that the higher the sigma, the less false positives but more true negatives and vice versa.

C

For sure that there arr some bad datas getting in. How muc they affect the result is depends on the threshold of our data system. They are unavoidable, since it can by lower by lowering (making more sensitive) our statistical threshold but this comes at the cost of potentially more omissions.

# problem 2 

In [ ]:
a = np.vstack((stats.norm.rvs( scale = 1, size = 100000), stats.norm.rvs( scale = 1, size = 100000)))
a.shape
fig, ax = plt.subplots(1, 1)
h = ax.hist2d(a[0,:],a[1,:],bins=100, density=True);
ax.set_aspect('equal', 'box')
plt.xlim([-3 , 3])
plt.ylim([-3 , 3])
plt.title("2D Histogram of positional uncertainty", fontsize = 24)
plt.ylabel("$\Delta$y arcseconds", fontsize = 18)
plt.xlabel("$\Delta$x arcseconds", fontsize = 18)
plt.colorbar(h[3], ax=ax)

1

2

What is the distance away from the origin at which the 2-dimensional Gaussian background might produce a position measurement that has a significance of $5\sigma$?

3

In [ ]:
dist_2d = stats.rayleigh(scale=1)
distance = dist_2d.ppf(dist.cdf(5))
pdf_2d = lambda r: r*np.exp(-r**2 / 2)
value, error = sp.integrate.quad(pdf_2d, distance, np.inf)
print(f"Distance for 5 sigma: {distance} arc-seconds")
print(f"Converting back to sigma: {dist.ppf(1 - value)}")

# problem 3

1

Is that safe to say that a moon is blocking cosmic rays from reaching earth if we have observed 6800 cosmic rays over 15 days? What is the probability of observing 6800 cosmic rays or within 15 days? Will the calculation based on the distribution of expected cosmic rays observed of 1 cosmic ray per minute?

2

What's the probability if we set mu = 1 (cosmic ray / minute) (60 minutes / hour) 8(hour / day) * 15 (days) = 7200, and k = P(k <= 6800) ?

In [ ]:
k = 6800
mu = 7200
prob = stats.poisson.cdf(k, mu)
print("Probability of 6800 rays or less: " + str(prob))

3

In [ ]:
sig = stats.norm.ppf(prob)
print("Sigma of 6800: " + str(sig))
print("Magnitude of sigma: " + str(abs(sig)))

the sigma value is less than 5